# Production

Putting a Python program to use requires moving it from a dev env to a production env. **Supporting disparate config like this can be a challenge**.

The goal is to *productionize* your Python programs and make them bulletproof while they're in use. Python has built-in modules that aid in hardening your programs.

## Item 54: Consider Module-Scoped Code to Configure Deployment Envs

A deployment env is a config in which your program runs. Every program has at least one deployment env, the **production env**.

Writing or modifying a program requires being able to run it on the computer you use for developing. The config of your **dev env** may be much different rom your prod env.

A good way to work around is to override parts of your program at startup time to provide different functionality depending on the deployment env. e.g. u could have two diff `__main__` files, one for prod and one for dev.

```python
# dev_main.py
TESTING = True
import db_connection
db = db_connection.Database()

# prod_main.py
TESTING = False
import db_connection
db = db_connection.Database()

# db_connection.py
import __main__

if __main__.TESTING:
    # Database = TestDatabase
else:
    # Database = RealDatabase
```

This approach can be used for more than working around external assumptions. e.g. if u know that ur program must work differently based on its host platform, you can inspect the `sys` module before defining top-level constructs in a module.

```python
# db_connection.py
import sys

if sys.platform.startswith('win32'):
    # Database = Win32Database
else:
    # Database = PosixDatabase
```


## Item 52: Use `repr` Strings for Debugging Output

## Item 53: Test Everything with `unittest`

Python doesn’t have static type checking. There’s nothing in the compiler that will ensure that your program will work when you run it. With Python you don’t know whether the functions your program calls will be defined at runtime, even when their existence is evident in the source code. This dynamic behavior is a blessing and a curse.

The large numbers of Python programmers out there say **it’s worth it because of the productivity gained from the resulting brevity and simplicity**. But most people have heard at least one horror story about Python in which a program encountered a boneheaded error at runtime.

Type safety isn't everything, you should always test your code, regardless of what language it's written in. However, I’ll admit that the big difference between Python and many other languages is that **the only way to have any confidence in a Python program is by writing tests**.

Luckily, the same dynamic features that prevent static type checking in Python also make it extremely easy to write tests for ur code.

**Having good tests actually makes it easier to modify Python code, not harder**.

The simplest way to write tests is to use the `unittest` built-in module.

```python
# utils.py
def to_str(data):
	if isinstance(data, str):
		return data
	elif isinstance(data, bytes):
		return data.decode('utf-8')
	else:
		raise TypeError('Must supply str or bytes, '
			'found: %r' % data)
```

```python
# test_utils.py
from unittest import TestCase, main
from utils import to_str


class UtilsTestCase(TestCase):
	def test_to_str_bytes(self):
		self.assertEqual('hello', to_str(b'hello'))

	def test_to_str_str(self):
		self.assertEqual('hello', to_str('hello'))

	def test_to_str_bad(self):
		self.assertRaises(TypeError, to_str, object())


if __name__ == '__main__':
	main()
```

I usually define one `TestCase` for each set of related tests. Sometimes I have one `TestCase` for each function that has many edge cases. Other times, a `TestCase` spans all functions in a single module. I’ll also create one `TestCase` for testing a single class and all of its methods.

When programs get complicated, you’ll want additional tests for verifying the interactions between your modules, instead of only testing code in isolation. This is the difference between **unit tests** and **integration tests**. In Python, it’s important to write both types of tests for exactly the same reason: You have no guarantee that your modules will actually work together unless you prove it.

See more info:
* `unittest.mock`
* `nose` package
* `pytest` package


## Item 57: Consider Interactive Debugging with pdb

## Item 58: Profile Before Optimizing

## Item 59: Use `tracemalloc` to Understand Memory Usage and Leaks